## Import Things

In [ ]:
import gpt_2_simple as gpt2
from big_phoney import BigPhoney
import numpy as np
import lyricsgenius as genius
import pandas as pd
import utils

import nlpaug.augmenter.word as naw

from nlpaug.util import Action
# model_name = "124M"
# model is saved into current directory under /models/124M/
# gpt2.download_gpt2(model_name=model_name) #need to run only once. comment out once done.

## Load models and outside functions

In [ ]:
#Initiate TensorFlow session, needs GPU
#Might need to start this inside the function so I can choose the right "run" a.k.a model
# from timeit import default_timer as timer
# print('import tensorflow')
# start = timer()
sess = gpt2.start_tf_sess()
# end = timer()
# print('Elapsed time: ' + str(end - start))

gpt2.load_gpt2(sess, run_name='run1')

#Load text augmentation model
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")

#For syllable counting
phoney = BigPhoney()

## Functions 

These will go in a Utils.py file 

In [ ]:
def get_target_lyrics(artist, title):
    
    api_key = 'Nc3VjqRNlEZiD9mqOQATDQ5PuZ4IjdgV13n7c39OIyNwsjuazZThQlQkdr_Hts4c'
    api = genius.Genius(api_key)
    api.remove_section_headers = True # Removes section headers like "Verse" and "Intro"
    
    #get song lyrics
    song = api.search_song(artist, title)
    
    #clean up the target lyrics
    target = song.lyrics
    target = target.replace("\u2005"," ")
    target = target.replace("\\"," ")
    target = target.replace("\\n"," ")
    target = target.replace("("," ")
    target = target.replace(")"," ")
    target = target.replace("\n\n","\n")
    target = target.replace("\n\n\n","\n")
    target = target.replace("2x","")
    #create list of lines
    target_lyrics = target.split("\n")
    print("Target lyrics:")
    print(target_lyrics)
    return target_lyrics

In [ ]:
# Later on I'll need to add the artist as a parameter, if I have 3, so it can load in the right finetuned model
# Figure out prefix
def generate_lyrics(): 
    
    gen_lyrics = gpt2.generate(sess, 
                     length=250,
                     temperature=0.9,
                     include_prefix=False,
                     #prefix="Deep in the night",
                     nsamples=1,
                     batch_size=1,
                     return_as_list=True)[0]
    
    gen_lyrics = gen_lyrics.split("\n")
    
    return gen_lyrics

In [28]:
def lyrics_to_list(file_name):
    with open(file_name, 'r') as file:
        data = file.read().split("====================")
        random_song = random.choice(data)
        gen_lyrics = random_song.split("\n")
        gen_lyrics = gen_lyrics[1:-1]
    #print(gen_lyrics)
    return gen_lyrics

In [33]:
lyrics_to_list('gpt2_gentext_slayer.txt')

['the past seven years, im lived to the fullest!',
 'dont know what im talking about',
 'im living my life with no memories',
 'no one can see',
 'i am the face of your concern',
 'i am the face of your wrath',
 'i am the scream that your mother goddess has conjured',
 'from the depths of your mind',
 '',
 'skeletons in my head',
 'anticipation the stimulation',
 'to kill the exhilaration',
 'close your eyes',
 'look deep in your soul',
 'step outside yourself',
 'and let your mind go',
 'frozen eyes stare deep in your mind as you die',
 'close your eyes',
 'and forget your name',
 'step outside yourself',
 'and let your thoughts drain',
 'as you go insane, go insane',
 'innate seed',
 'to watch you bleed',
 'a demanding physical need',
 'desecrated eviscerated',
 'times perpetuated',
 'close your eyes',
 'look deep in your soul',
 'step outside yourself',
 'and let your mind go',
 'frozen eyes stare deep in your mind as you die',
 'close your eyes',
 'and forget your name',
 'step out

In [13]:
def count_syls(text):
    
    schema = []
    
    for line in text:
        syls = phoney.count_syllables(line)
        schema.append(syls)
        #print(syls,line)
        
    return schema

KeyboardInterrupt: 

#### The magical lyrics fitting function

In [ ]:
from flaskexample.utils import lyrics_to_list, get_target_lyrics, fit_lyrics

In [ ]:
#This function uses the count_syls function above to keep track of num of syllables in each line
def fit_lyrics(gen_lyrics, target_lyrics):
    aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")
   # model._make_predict_function()
    phoney = BigPhoney()
    def count_syls(text):
    
        schema = []
    
        for line in text:
            syls = phoney.count_syllables(line)
            schema.append(syls)
            #print(syls,line)
        
        return schema
    
    gen_schema = count_syls(gen_lyrics)
    target_schema = count_syls(target_lyrics)
    print(target_schema)
    #make generated lyrics same length as target lyrics
    target_len = len(target_schema)
    del gen_schema[target_len:]
    del gen_lyrics[target_len:]
#    print(gen_schema)
    #print(target_schema)
    new_lyrics = []
    loop = 1
    for num, line in enumerate(gen_lyrics):
        #print(num)
        if (gen_schema[num] == target_schema[num]): 
            new_lyrics.append(line)
        elif (gen_schema[num] < target_schema[num]):
            syls = gen_schema[num]
            while syls < target_schema[num]:
                original_line = line
                line = aug.augment(line)
                syls = phoney.count_syllables(line)
#                print("augmented:")
#                print(line)
                #In case we overshoot (add too many syllables)
                if syls > target_schema[num]:
#                    print("Oops we overshot")
#                    print(syls)
                    #target = target_schema[num]
                    line = original_line
                    syls = phoney.count_syllables(line)
#                print("gen")
#                print(gen_schema[num])
#                print("target")
#                print(target_schema[num])
#                print("syls:")
#                print(syls)
#                print(line)
                #Just so we don't get stuck in the loop forever
                loop = loop+1
                if loop > 50:
                    syls = target_schema[num]
            new_lyrics.append(line)
        elif (gen_schema[num] > target_schema[num]):
            syls = gen_schema[num]
            words = line.split(" ")
            while syls > target_schema[num]:
                del words[-1]
                new_line = ' '.join(words)
                syls = phoney.count_syllables(new_line)
                #In case too many syllables are deleted
                if syls < target_schema[num]:
                    #print("Oops, deleted too many")
                    new_line = aug.augment(new_line)
                    syls = phoney.count_syllables(new_line)
                    #print(syls)
                #print(syls)
            new_lyrics.append(new_line)
    #print("New lyrics:")
    #print(new_lyrics)
    
    return new_lyrics

## Time to test everything

In [ ]:
artist = "Idina Menzel"
title = "Let it go"
target_lyrics = get_target_lyrics(artist, title)

In [ ]:
gen_lyrics = generate_lyrics()

In [ ]:
fit_lyrics(gen_lyrics, target_lyrics)

In [ ]:
print(target_lyrics)

In [ ]:
print(gen_lyrics)

In [ ]:
print(new_lyrics)

In [ ]:
print(target_schema)
print(gen_schema)

In [ ]:
from timeit import default_timer as timer
print('import tensorflow')
start = timer()
import tensorflow
end = timer()
print('Elapsed time: ' + str(end - start))

In [ ]:
sess.close()